In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import numpy as np
import os,sys,inspect

current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir)
import edhec_risk_kit as erk

import ipywidgets as widgets
from IPython.display import display

In [12]:
import matplotlib.pyplot as plt
import numpy as np

def show_cppi(n_scenarios=50, mu=0.07, sigma=0.15, m=3, floor=0., riskfree_rate=0.03, steps_per_year=12, y_max=100):
    """
    Plot the results of a Monte Carlo Simulation of CPPI
    """
    start = 100
    sim_rets = erk.gbm(n_scenarios=n_scenarios, mu=mu, sigma=sigma, prices=False, steps_per_year=steps_per_year)
    risky_r = pd.DataFrame(sim_rets)
    # run the "back"-test
    btr = erk.run_cppi(risky_r=pd.DataFrame(risky_r),riskfree_rate=riskfree_rate,m=m, start=start, floor=floor)
    wealth = btr["Wealth"]

    # calculate terminal wealth stats
    y_max=wealth.values.max()*y_max/100
    terminal_wealth = wealth.iloc[-1]
    
    tw_mean = terminal_wealth.mean()
    tw_median = terminal_wealth.median()
    failure_mask = np.less(terminal_wealth, start*floor)
    n_failures = failure_mask.sum()
    p_fail = n_failures/n_scenarios

    tw_max = terminal_wealth.max()
    tw_min = terminal_wealth.min()
    tw_difference = tw_max - tw_min

    # expected short fall
    e_shortfall = np.dot(terminal_wealth-start*floor, failure_mask)/n_failures if n_failures > 0 else 0.0

    # Plot!
    fig, (wealth_ax, hist_ax) = plt.subplots(nrows=1, ncols=2, sharey=True, gridspec_kw={'width_ratios':[3,2]}, figsize=(24, 9))
    plt.subplots_adjust(wspace=0.0)
    
    wealth.plot(ax=wealth_ax, legend=False, alpha=0.3, color="indianred")
    wealth_ax.axhline(y=start, ls=":", color="black")
    wealth_ax.axhline(y=start*floor, ls="--", color="red")
    wealth_ax.set_ylim(top=y_max)
    
    terminal_wealth.plot.hist(ax=hist_ax, bins=50, ec='w', fc='indianred', orientation='horizontal')
    hist_ax.axhline(y=start, ls=":", color="black")
    hist_ax.axhline(y=tw_mean, ls=":", color="blue")
    hist_ax.axhline(y=tw_median, ls=":", color="purple")
    hist_ax.annotate(f"Mean: ${int(tw_mean)}", xy=(.7, .9),xycoords='axes fraction', fontsize=24)
    hist_ax.annotate(f"Median: ${int(tw_median)}", xy=(.7, .85),xycoords='axes fraction', fontsize=24)
    hist_ax.annotate(f"WSS: ${int(tw_min)}", xy=(.7, .80),xycoords='axes fraction', fontsize=24)
    if (floor > 0.01):
        hist_ax.axhline(y=start*floor, ls="--", color="red", linewidth=3)
        hist_ax.annotate(f"Violations: {n_failures} ({p_fail*100:2.2f}%)\nE(shortfall)=${e_shortfall:2.2f}", xy=(.7, .7), xycoords='axes fraction', fontsize=24)

cppi_controls = widgets.interactive(show_cppi,
                                   n_scenarios=widgets.IntSlider(min=1, max=1000, step=5, value=50), 
                                   mu=(0., +.2, .01),
                                   sigma=(0, .45, .05),
                                   floor=(0, 2, .1),
                                   m=(1, 5, .5),
                                   riskfree_rate=(0, .05, .01),
                                   steps_per_year=widgets.IntSlider(min=1, max=12, step=1, value=12,
                                                          description="Rebals/Year"),
                                   y_max=widgets.IntSlider(min=0, max=100, step=1, value=100,
                                                          description="Zoom Y Axis")
)
display(cppi_controls)

interactive(children=(IntSlider(value=50, description='n_scenarios', max=1000, min=1, step=5), FloatSlider(val…

In [74]:
def q3_increase_mu(n_scenarios=500, mu=0.00, sigma=0.15, m=3, floor=0.7, riskfree_rate=0.03, steps_per_year=12, y_max=100):
    start = 100    
    mu = mu/100
    sim_rets = erk.gbm(n_scenarios=n_scenarios, mu=mu, sigma=sigma, prices=False, steps_per_year=steps_per_year)
    risky_r = pd.DataFrame(sim_rets)
    # run the "back"-test
    btr = erk.run_cppi(risky_r=pd.DataFrame(risky_r),riskfree_rate=riskfree_rate,m=m, start=start, floor=floor)
    wealth = btr["Wealth"]
    
    # calculate terminal wealth stats
    y_max=wealth.values.max()*y_max/100
    terminal_wealth = wealth.iloc[-1]
    
    tw_mean = terminal_wealth.mean()
    tw_median = terminal_wealth.median()
    tw_max = terminal_wealth.max()
    tw_min = terminal_wealth.min()
    tw_diff = tw_max - tw_min
    failure_mask = np.less(terminal_wealth, start*floor)
    n_failures = failure_mask.sum()
    p_fail = n_failures/n_scenarios

    e_shortfall = np.dot(terminal_wealth-start*floor, failure_mask)/n_failures if n_failures > 0 else 0.0

    return {"MU": mu, "Mean": tw_mean, "Median": tw_median, "Diff": tw_diff, "Worst case": tw_min, "N Failures": n_failures, "Exp. Shortfall": e_shortfall}

### Question 1

In [75]:
rows_list = []
for n_scenarios in range(450, 500):
    result = q3_increase_mu(n_scenarios=n_scenarios, floor=0)
    rows_list.append(result)
pd.DataFrame(rows_list)

,MU,Mean,Median,Diff,Worst case,N Failures,Exp. Shortfall
0,0.0,98.511100,85.932129,342.029924,24.403063,0,0.0
1,0.0,98.876311,88.752786,287.284723,19.866136,0,0.0
2,0.0,102.125761,93.852033,363.193458,27.908043,0,0.0
3,0.0,99.979737,88.744229,398.616623,18.580741,0,0.0
4,0.0,94.799140,85.445318,288.081693,18.391444,0,0.0
5,0.0,97.369105,90.202211,280.504120,18.290706,0,0.0
6,0.0,99.715031,88.409942,336.932324,23.950372,0,0.0
7,0.0,101.494227,86.905313,310.070194,26.798568,0,0.0
8,0.0,97.352327,89.135032,381.418944,24.099122,0,0.0
9,0.0,95.436614,86.528752,280.313462,14.528466,0,0.0


In [81]:
rows_list = []
riskfree_rates = [.0075, .01, .015, .02, .025, .03, .05, .1]
n_scenarioss = [500, 510, 520, 530, 540, 550]
for riskfree_rate in riskfree_rates:
    for n_scenarios in n_scenarioss:
        result = q3_increase_mu(riskfree_rate=riskfree_rate, floor=0, n_scenarios=n_scenarios)
        rows_list.append({"n_scenarios": n_scenarios, "riskfree_rate": riskfree_rate, "diff": result["Diff"]})
frame = pd.DataFrame(rows_list)
frame[["riskfree_rate", "diff"]].groupby("riskfree_rate").mean()

,diff
riskfree_rate,
0.0075,336.890646
0.0100,299.109117
0.0150,353.039691
0.0200,365.182253
0.0250,391.809322
0.0300,316.941441
0.0500,341.954249
0.1000,321.984201


## Increase mu 0 -> 0.2
### Question 3

In [44]:
rows_list = []
for mu in range(0, 21):
    result = q3_increase_mu(mu=mu)
    rows_list.append(result)
pd.DataFrame(rows_list)

,MU,Mean,Median,Worst case,N Failures
0,0.00,108.189395,91.348853,72.754900,0
1,0.01,114.249051,95.346833,73.265659,0
2,0.02,125.889103,107.354306,73.256806,0
3,0.03,133.802884,112.537454,74.262638,0
4,0.04,152.167037,130.058857,74.565502,0
5,0.05,160.985936,140.687104,72.489478,0
6,0.06,178.287132,155.004451,74.593786,0
7,0.07,199.795164,176.796738,74.027974,0
8,0.08,213.489798,188.415711,76.955991,0
9,0.09,231.193478,203.997720,74.719331,0


## All other things being equal, which of these changes will cause an INCREASE in floor violations
### Question 4

In [47]:
iter_range = [(1, 0.1), (2, 0.15), (3, 0.20), (4, 0.30), (5, 0.45)]
rows_list = []
for (m, sigma) in iter_range:
    result = q3_increase_mu(m=m, sigma=sigma)
    rows_list.append(result)
pd.DataFrame(rows_list)

,MU,Mean,Median,Worst case,N Failures
0,0.0,120.803028,119.498266,93.251023,0
1,0.0,109.304242,99.002660,76.314574,0
2,0.0,108.650606,85.088258,71.776153,0
3,0.0,106.762662,75.297663,69.267405,3
4,0.0,107.375505,70.998617,59.410081,79


In [48]:
iter_range = [(5, 0.1), (4, 0.15), (3, 0.20), (2, 0.30), (1, 0.45)]
rows_list = []
for (m, sigma) in iter_range:
    result = q3_increase_mu(m=m, sigma=sigma)
    rows_list.append(result)
pd.DataFrame(rows_list)

,MU,Mean,Median,Worst case,N Failures
0,0.0,103.256092,96.235638,72.249469,0
1,0.0,106.406088,87.812868,72.057803,0
2,0.0,107.365624,84.874396,71.654144,0
3,0.0,111.393287,84.305056,71.824017,0
4,0.0,123.054829,98.997117,72.477057,0


In [49]:
iter_range = [(1, 0.45), (2, 0.30), (3, 0.20), (4, 0.15), (5, 0.1)]
rows_list = []
for (m, sigma) in iter_range:
    result = q3_increase_mu(m=m, sigma=sigma)
    rows_list.append(result)
pd.DataFrame(rows_list)

,MU,Mean,Median,Worst case,N Failures
0,0.0,117.875522,96.932853,73.545611,0
1,0.0,117.273034,86.386620,71.595748,0
2,0.0,110.066332,83.931601,71.337551,0
3,0.0,103.157207,84.700198,71.769810,0
4,0.0,106.355675,98.874139,72.700105,0


## All other things being equal, which of these changes will cause an INCREASE in floor violations
### Question 5, 6

In [61]:
iter_range = [(1, 12), (2, 9), (3, 6), (4, 3), (5, 1)]
rows_list = []
for (m, steps_per_year) in iter_range:
    result = q3_increase_mu(m=m, steps_per_year=steps_per_year)
    rows_list.append(result)
pd.DataFrame(rows_list)

,MU,Mean,Median,Worst case,N Failures,Exp. Shortfall
0,0.0,121.252524,118.263186,88.456001,0,0.000000
1,0.0,108.546789,97.056000,75.488968,0,0.000000
2,0.0,103.293791,86.452233,70.831253,0,0.000000
3,0.0,102.372237,80.087202,66.586649,5,-1.823885
4,0.0,99.958138,77.219824,44.430837,109,-4.351080


In [59]:
iter_range = [(5, 1), (4, 3), (3, 6), (2, 9), (1, 12)]
rows_list = []
for (m, steps_per_year) in iter_range:
    result = q3_increase_mu(m=m, steps_per_year=steps_per_year)
    rows_list.append(result)
pd.DataFrame(rows_list)

,MU,Mean,Median,Worst case,N Failures,Exp. Shortfall
0,0.0,97.016447,75.449519,43.541153,132,-4.165308
1,0.0,102.133956,83.772404,69.088632,2,-0.879365
2,0.0,104.918969,88.533601,71.690011,0,0.000000
3,0.0,108.589151,98.808569,74.009254,0,0.000000
4,0.0,122.517651,117.691745,88.819381,0,0.000000


In [60]:
iter_range = [(1, 1), (2, 3), (3, 6), (4, 9), (5, 12)]
rows_list = []
for (m, steps_per_year) in iter_range:
    result = q3_increase_mu(m=m, steps_per_year=steps_per_year)
    rows_list.append(result)
pd.DataFrame(rows_list)

,MU,Mean,Median,Worst case,N Failures,Exp. Shortfall
0,0.0,102.238599,98.671116,77.272680,0,0.0
1,0.0,104.215317,94.249293,72.150792,0,0.0
2,0.0,102.969252,84.850688,71.422099,0,0.0
3,0.0,104.761144,86.552225,71.117299,0,0.0
4,0.0,104.857691,86.260986,70.983755,0,0.0


## Parameter changes that increase the probability of floor violations will also tend to increase the Expected Shortfall. This statement is:
### Question 7

In [62]:
# Based on this I gave an incorrect "False" answer
iter_range = [0.01, 0.015, 0.02, 0.025, 0.030, 0.305, 0.04]
rows_list = []
for riskfree_rate in iter_range:
    result = q3_increase_mu(riskfree_rate=riskfree_rate)
    rows_list.append(result)
pd.DataFrame(rows_list)

,MU,Mean,Median,Worst case,N Failures,Exp. Shortfall
0,0.0,105.584546,85.331163,70.900429,0,0.0
1,0.0,102.385182,86.112227,70.907288,0,0.0
2,0.0,108.217697,89.674947,71.361268,0,0.0
3,0.0,103.712936,89.111105,72.091099,0,0.0
4,0.0,105.884504,89.721875,72.932434,0,0.0
5,0.0,131.129218,113.039370,82.912743,0,0.0
6,0.0,108.670043,92.817872,73.517658,0,0.0


It doesn't make sense

False